<a href="https://colab.research.google.com/github/PremKarhale/Full-stack-using-Agentic-AI-/blob/main/AI_powered_fitness_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI POWERED FITNESS APP PLANNER**

In [2]:
!pip install -qU langchain_google_genai langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google

# **Tool** - for calculating metrics (BMI,BMR,required calaroies)

In [28]:
weight=int(input('Enter your weight in kg: '))
height_m=float(input('Enter your height in meters: '))
age=int(input('Enter your age:'))
gender=input('Enter your gender: ')
activity_level=input('Enter your activity level form sedentary/light/moderate/active : ')
Fitness_goal=input('Enter your Fitness goal from lose weight/gain weight/Maintain weight & build muscles: ')

Enter your weight in kg: 60
Enter your height in meters: 1.75
Enter your age:21
Enter your gender: male
Enter your activity level form sedentary/light/moderate/active : active
Enter your Fitness goal from lose weight/gain weight/Maintain weight & build muscles: Maintain weight & build muscles


In [29]:
# tool for calculating BMI or BMR
from ast import Call
from langchain_core.tools import tool

@tool
def BMI_Calculator(
    weight: float,
    height_m: float,
    age: int,
    gender: str,
    activity_level: str,
    Fitness_goal: str
) -> dict:
    """
    Calculates BMI, category, BMR, and maintenance calories
    """

    # ---------- BMI ----------
    bmi = weight / (height_m ** 2)

    if bmi < 18.5:
        bmi_category = "Underweight"
    elif bmi < 25:
        bmi_category = "Normal"
    elif bmi < 30:
        bmi_category = "Overweight"
    else:
        bmi_category = "Obesity"

    # ---------- BMR ----------
    height_cm = height_m * 100

    if gender.lower() == 'male':
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) + 5
    else:
        bmr = (10 * weight) + (6.25 * height_cm) - (5 * age) - 161

    # ---------- Activity Multiplier ----------
    activity_factors = {
        'sedentary': 1.2,
        'light': 1.375,
        'moderate': 1.55,
        'active': 1.725
    }

    factor = activity_factors.get(activity_level.lower(), 1.2)
    maintenance_calories = bmr * factor

    # ---------- Weight Goal Adjustment ----------
    if Fitness_goal.lower() == 'lose weight':
        maintenance_calories -= 500
    elif Fitness_goal.lower() == 'gain weight':
        maintenance_calories += 500

    # ---------- Final Output ----------
    return {
        "BMI": round(bmi, 2),
        "BMI_Category": bmi_category,
        "BMR": round(bmr, 2),
        "Daily_calories as per goal": round(maintenance_calories, 2)
    }



In [30]:
from google.colab import userdata
gemini_api=userdata.get('GEMINI_API_KEY')

# Tool Binding

In [31]:
# llm testing
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=gemini_api)

prompt=PromptTemplate(template='this are user details {weight},{height_m},{age},{gender},{activity_level},{Fitness_goal} based on this usr data give me the desired results',input_variables=[weight,height_m,age,gender,activity_level,Fitness_goal])

llm_with_tool = llm.bind(tools=[BMI_Calculator])

chain = prompt | llm_with_tool




- llm tells us with tool to use with kargs , we have to invoke the desired tool by overself

In [32]:
response=chain.invoke({'weight':weight,'height_m':height_m,'age':age,'gender':gender,'activity_level':activity_level,'Fitness_goal':Fitness_goal})

 # Tool Execution

In [33]:
print(response.tool_calls[0])

{'name': 'BMI_Calculator', 'args': {'height_m': 1.75, 'Fitness_goal': 'Maintain weight & build muscles', 'activity_level': 'active', 'weight': 60, 'age': 21, 'gender': 'male'}, 'id': '90f25985-477f-482c-8eba-2fe45d4c7319', 'type': 'tool_call'}


In [34]:
metrics = BMI_Calculator.invoke(response.tool_calls[0])
print(metrics.content)

{"BMI": 19.59, "BMI_Category": "Normal", "BMR": 1593.75, "Daily_calories as per goal": 2749.22}


generate the 7 day workout and diet plan / chains or RunnableParallel

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
api_key=userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',google_api_key=api_key)

In [36]:
#workout chain

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate(template="""
You are a certified fitness trainer and health coach.

Based on the following user information:
- Health Metrics: {health_metrics}
- curr_weight: {weight}
- Activity Level: {activity_level}
- Fitness Goal: {fitness_goal}

Generate a detailed, realistic 7-day workout plan.

Requirements:
- Include daily workout split (Day 1 to Day 7)
- Mention warm-up, main workout, and cool-down
- Adjust intensity according to activity level
- Keep exercises safe based on health metrics
- Include rest or active recovery if needed
- Use simple, clear language

Output format:
Day 1:
- Warm-up:
- Workout:
- Cool-down:

(Repeat for all 7 days)
""",input_variables=[
        "health_metrics",
        "weight",
        "activity_level",
        "fitness_goal"
    ])

Workout_chain = prompt | llm | StrOutputParser()

In [37]:
# Diet chain
prompt = PromptTemplate(template=
                        """
You are a certified fitness trainer and health coach.

Based on the following user information:
- Health Metrics: {health_metrics}
- curr weight: {weight}
- Activity Level: {activity_level}
- Fitness Goal: {fitness_goal}

Generate a detailed, realistic 7-day diet plan with the following considerations:
1. Include **balanced meals** with a mix of **protein**, **healthy fats**, and **complex carbohydrates**.
2. Account for **calories** appropriate for the user's weight goal (whether it's fat loss, muscle gain, or maintenance).
3. Adjust the meals according to the user's **activity level** (low, moderate, high).
4. Ensure the diet supports the **fitness goal** (e.g., endurance, strength, weight loss, muscle gain).
5. Provide options for **snacks** that are easy to prepare and fit within the dietary needs.

The plan should include **meal details** for each day, such as:

- Breakfast
- Snack 1
- Lunch
- Snack 2
- Dinner
- Evening snack (if needed)

Please ensure the food options are **realistic**, **easy to prepare**, and nutritionally balanced.
"""
 ,input_variables=[
        "health_metrics",
        "weight",
        "activity_level",
        "fitness_goal"
    ])
diet_chain = prompt | llm | StrOutputParser()

In [40]:
from langchain_core.runnables import RunnableParallel
resp=RunnableParallel({
    "workout_plan":Workout_chain,
    "diet_plan":diet_chain
})
result = resp.invoke({
    "health_metrics":metrics,
    "weight":weight,
    "activity_level":activity_level,
    "fitness_goal":Fitness_goal
})



In [41]:
print("These are your HEALTH METRICS :",metrics.content)

These are your HEALTH METRICS : {"BMI": 19.59, "BMI_Category": "Normal", "BMR": 1593.75, "Daily_calories as per goal": 2749.22}


In [42]:
print(result['workout_plan'])

Here is a detailed, realistic 7-day workout plan designed to help you maintain your weight and build muscle, tailored to your active lifestyle and normal BMI. This plan incorporates strength training, active recovery, and proper rest to support muscle growth and overall fitness.

Remember to listen to your body, maintain proper form for all exercises, and adjust weights as needed to ensure you're challenged but can complete the sets with good technique. Hydration and nutrition are also key for success!

---

**General Guidelines for Each Day:**

*   **Warm-up:** Aim for 5-10 minutes of light cardio (e.g., brisk walking, light jogging, cycling) followed by dynamic stretches (e.g., arm circles, leg swings, torso twists, bodyweight squats).
*   **Cool-down:** Aim for 5-10 minutes of static stretching, holding each stretch for 20-30 seconds. Focus on the muscles worked during your main workout.
*   **Rep Range:** For muscle building, aim for 3-4 sets of 8-12 repetitions for most strength e

In [39]:
print(result['diet_plan'])

Hello there! As your certified fitness trainer and health coach, I'm excited to help you fuel your fitness journey. Based on your current weight, active lifestyle, and goal to maintain weight while building muscle, your daily calorie target is approximately **2750 calories**. This plan is designed to provide ample protein for muscle repair and growth, complex carbohydrates for sustained energy, and healthy fats for overall well-being, all while being realistic and easy to follow.

Remember to listen to your body, adjust portion sizes as needed based on your hunger and energy levels, and stay well-hydrated throughout the day (aim for 2.5-3.5 liters of water).

---

### **7-Day Muscle Building & Maintenance Diet Plan (Approx. 2750 kcal/day)**

**Key Principles:**
*   **Protein Power:** Each meal and most snacks include a good source of protein.
*   **Complex Carbs:** Prioritize whole grains, fruits, and vegetables for sustained energy.
*   **Healthy Fats:** Include sources like avocados,